In [1]:
library(tidyverse)
library(dplyr)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.4     ✔ purrr   0.3.4
✔ tibble  3.1.2     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [2]:
BiocManager::install("Biobase")
BiocManager::install("biomaRt")

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: http://cran.rstudio.com/


Bioconductor version 3.12 (BiocManager 1.30.15), R 4.0.5 (2021-03-31)

Installing package(s) 'Biobase'

Old packages: 'ade4', 'adegenet', 'admisc', 'afex', 'agricolae', 'akima',
  'Amelia', 'AMR', 'aqp', 'arrow', 'arsenal', 'arules', 'arulesViz',
  'autocogs', 'autostsm', 'baguette', 'bayesplot', 'bayestestR', 'bbmle',
  'beeswarm', 'bestNormalize', 'bibliometrix', 'biclust', 'bigrquery',
  'BiocManager', 'biotools', 'biwavelet', 'blob', 'BMA', 'bnstruct',
  'bookdown', 'brms', 'broom', 'broom.mixed', 'bslib', 'bsts', 'butcher',
  'C50', 'cachem', 'car', 'causaleffect', 'CausalImpact', 'causalweight',
  'cdata', 'circlize', 'cli', 'clifro', 'clock', 'ClusterR', 'clustMixType',
  'colorspace', 'compositions', 'coronavirus', 'corrplot', 'countrycode',
  'cpp11', 'credentials', 'Cubist', 'cvms', 'd3r', 'DALEX', 'datamods',
  'DEo

In [3]:
library(Biobase)
library(biomaRt)

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Welcome to Biocond

# Mapping Gene Symbols

In [4]:
expr.mat <- read.csv("../input/aml-fab/GSE147515_FAB.csv")

In [5]:
head(expr.mat)

,X,ID,GSM1254265.CEL,GSM1254267.CEL,GSM1254268.CEL,GSM1254269.CEL,GSM1254270.CEL,GSM1254271.CEL,GSM1254272.CEL,GSM1254273.CEL,⋯,GSM726248.CEL,GSM726249.CEL,GSM726251.CEL,GSM726252.CEL,GSM726253.CEL,GSM726256.CEL,GSM726257.CEL,GSM726258.CEL,GSM726259.CEL,GSM726260.CEL
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,1053_at,7.537840,6.800871,6.752669,5.929756,7.353971,6.496232,6.716364,7.407105,⋯,5.526605,7.605745,7.701945,7.096387,6.600813,6.563042,6.671309,5.930616,6.119624,6.554417
2,2,117_at,4.679401,4.777950,4.360962,4.509553,4.720435,7.669224,5.432612,4.435121,⋯,5.071652,4.396333,5.979429,4.177423,4.290783,5.271843,5.841040,7.307721,6.138064,4.855038
3,3,121_at,6.798042,7.370561,7.043800,7.132901,7.272406,6.851993,7.215697,7.225802,⋯,6.637921,7.564590,6.726327,7.424036,7.026728,6.710751,7.198889,6.996386,6.894396,7.371354
4,4,1255_g_at,2.704549,2.500655,2.644052,2.712234,2.861128,2.913571,2.820453,2.752378,⋯,2.918437,2.650575,2.795106,2.576289,2.807488,2.784839,2.746156,2.694078,3.009329,2.856732
5,5,1316_at,4.544424,5.050274,4.988216,4.862662,5.144750,4.837291,4.871102,4.981282,⋯,6.408416,4.846550,5.505493,5.088999,5.112156,5.743054,5.497159,5.444714,5.167476,5.273593
6,6,1320_at,3.734027,3.630663,3.587355,4.006397,3.762099,3.608655,3.754732,3.931028,⋯,3.931349,3.784949,3.751392,3.702852,3.803101,3.597913,3.614316,3.799544,3.681782,3.887242


In [6]:
mart <- useMart("ENSEMBL_MART_ENSEMBL", host="uswest.ensembl.org")

In [7]:
mart <- useDataset("hsapiens_gene_ensembl", mart)

In [8]:
annotLookup <- getBM(
    mart=mart,
    attributes=c(
        "affy_hg_u133_plus_2",
        "ensembl_gene_id",
        "gene_biotype",
        "external_gene_name"
    ),
    filter="affy_hg_u133_plus_2",
    values = expr.mat$ID,
    uniqueRows=TRUE
)

In [9]:
head(annotLookup)

,affy_hg_u133_plus_2,ensembl_gene_id,gene_biotype,external_gene_name
,<chr>,<chr>,<chr>,<chr>
1,1553551_s_at,ENSG00000198888,protein_coding,MT-ND1
2,1553551_s_at,ENSG00000210100,Mt_tRNA,MT-TI
3,1553551_s_at,ENSG00000210112,Mt_tRNA,MT-TM
4,1553551_s_at,ENSG00000198763,protein_coding,MT-ND2
5,1553538_s_at,ENSG00000198804,protein_coding,MT-CO1
6,1553569_at,ENSG00000198804,protein_coding,MT-CO1


In [10]:
indicesLookup <- match(expr.mat$ID, annotLookup$affy_hg_u133_plus_2)

In [11]:
table(expr.mat$ID == annotLookup[indicesLookup,'affy_hg_u133_plus_2'])


 TRUE 
37981 

In [12]:
annotLookup$affy_x_gene <- ifelse(annotLookup$external_gene_name == "",
                                 annotLookup$affy_hg_u133_plus_2,
                                  annotLookup$external_gene_name
                                 )

In [13]:
head(annotLookup)

,affy_hg_u133_plus_2,ensembl_gene_id,gene_biotype,external_gene_name,affy_x_gene
,<chr>,<chr>,<chr>,<chr>,<chr>
1,1553551_s_at,ENSG00000198888,protein_coding,MT-ND1,MT-ND1
2,1553551_s_at,ENSG00000210100,Mt_tRNA,MT-TI,MT-TI
3,1553551_s_at,ENSG00000210112,Mt_tRNA,MT-TM,MT-TM
4,1553551_s_at,ENSG00000198763,protein_coding,MT-ND2,MT-ND2
5,1553538_s_at,ENSG00000198804,protein_coding,MT-CO1,MT-CO1
6,1553569_at,ENSG00000198804,protein_coding,MT-CO1,MT-CO1


In [14]:
newnames <- paste(annotLookup[indicesLookup, "affy_x_gene"],
                c(1:length(indicesLookup)),
                sep="_")

In [15]:
head(expr.mat)

,X,ID,GSM1254265.CEL,GSM1254267.CEL,GSM1254268.CEL,GSM1254269.CEL,GSM1254270.CEL,GSM1254271.CEL,GSM1254272.CEL,GSM1254273.CEL,⋯,GSM726248.CEL,GSM726249.CEL,GSM726251.CEL,GSM726252.CEL,GSM726253.CEL,GSM726256.CEL,GSM726257.CEL,GSM726258.CEL,GSM726259.CEL,GSM726260.CEL
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,1053_at,7.537840,6.800871,6.752669,5.929756,7.353971,6.496232,6.716364,7.407105,⋯,5.526605,7.605745,7.701945,7.096387,6.600813,6.563042,6.671309,5.930616,6.119624,6.554417
2,2,117_at,4.679401,4.777950,4.360962,4.509553,4.720435,7.669224,5.432612,4.435121,⋯,5.071652,4.396333,5.979429,4.177423,4.290783,5.271843,5.841040,7.307721,6.138064,4.855038
3,3,121_at,6.798042,7.370561,7.043800,7.132901,7.272406,6.851993,7.215697,7.225802,⋯,6.637921,7.564590,6.726327,7.424036,7.026728,6.710751,7.198889,6.996386,6.894396,7.371354
4,4,1255_g_at,2.704549,2.500655,2.644052,2.712234,2.861128,2.913571,2.820453,2.752378,⋯,2.918437,2.650575,2.795106,2.576289,2.807488,2.784839,2.746156,2.694078,3.009329,2.856732
5,5,1316_at,4.544424,5.050274,4.988216,4.862662,5.144750,4.837291,4.871102,4.981282,⋯,6.408416,4.846550,5.505493,5.088999,5.112156,5.743054,5.497159,5.444714,5.167476,5.273593
6,6,1320_at,3.734027,3.630663,3.587355,4.006397,3.762099,3.608655,3.754732,3.931028,⋯,3.931349,3.784949,3.751392,3.702852,3.803101,3.597913,3.614316,3.799544,3.681782,3.887242


In [16]:
expr.mat$name <- annotLookup[indicesLookup, "affy_x_gene"]

In [17]:
head(expr.mat)

,X,ID,GSM1254265.CEL,GSM1254267.CEL,GSM1254268.CEL,GSM1254269.CEL,GSM1254270.CEL,GSM1254271.CEL,GSM1254272.CEL,GSM1254273.CEL,⋯,GSM726249.CEL,GSM726251.CEL,GSM726252.CEL,GSM726253.CEL,GSM726256.CEL,GSM726257.CEL,GSM726258.CEL,GSM726259.CEL,GSM726260.CEL,name
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,1,1053_at,7.537840,6.800871,6.752669,5.929756,7.353971,6.496232,6.716364,7.407105,⋯,7.605745,7.701945,7.096387,6.600813,6.563042,6.671309,5.930616,6.119624,6.554417,RFC2
2,2,117_at,4.679401,4.777950,4.360962,4.509553,4.720435,7.669224,5.432612,4.435121,⋯,4.396333,5.979429,4.177423,4.290783,5.271843,5.841040,7.307721,6.138064,4.855038,HSPA6
3,3,121_at,6.798042,7.370561,7.043800,7.132901,7.272406,6.851993,7.215697,7.225802,⋯,7.564590,6.726327,7.424036,7.026728,6.710751,7.198889,6.996386,6.894396,7.371354,PAX8
4,4,1255_g_at,2.704549,2.500655,2.644052,2.712234,2.861128,2.913571,2.820453,2.752378,⋯,2.650575,2.795106,2.576289,2.807488,2.784839,2.746156,2.694078,3.009329,2.856732,GUCA1A
5,5,1316_at,4.544424,5.050274,4.988216,4.862662,5.144750,4.837291,4.871102,4.981282,⋯,4.846550,5.505493,5.088999,5.112156,5.743054,5.497159,5.444714,5.167476,5.273593,THRA
6,6,1320_at,3.734027,3.630663,3.587355,4.006397,3.762099,3.608655,3.754732,3.931028,⋯,3.784949,3.751392,3.702852,3.803101,3.597913,3.614316,3.799544,3.681782,3.887242,PTPN21


In [18]:
tail(expr.mat)

,X,ID,GSM1254265.CEL,GSM1254267.CEL,GSM1254268.CEL,GSM1254269.CEL,GSM1254270.CEL,GSM1254271.CEL,GSM1254272.CEL,GSM1254273.CEL,⋯,GSM726249.CEL,GSM726251.CEL,GSM726252.CEL,GSM726253.CEL,GSM726256.CEL,GSM726257.CEL,GSM726258.CEL,GSM726259.CEL,GSM726260.CEL,name
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
42308,42308,91684_g_at,6.766378,6.301067,6.942578,7.182671,6.619921,6.066498,6.505410,6.035226,⋯,6.528560,6.719971,6.890480,7.020884,6.317323,6.477495,6.103140,5.734245,6.278673,EXOSC4
42309,42309,91703_at,6.747648,6.676374,6.881059,6.822209,7.069263,8.423341,6.621008,6.776445,⋯,7.224058,8.298455,7.084531,6.432603,7.648241,7.355685,8.489993,6.966807,6.403711,EHBP1L1
42310,42310,91816_f_at,3.408123,3.183218,2.989174,3.100119,3.051995,3.283360,3.096775,3.421928,⋯,4.308559,4.362729,3.832156,4.822119,3.246183,3.405585,3.209524,3.436144,3.152048,MEX3D
42311,42311,91826_at,5.018403,4.752440,4.660150,4.963589,4.880585,5.415532,4.880591,5.179232,⋯,5.615756,5.725632,5.402862,5.045892,5.154306,4.901181,5.079040,5.071480,4.677314,EPS8L1
42312,42312,91920_at,5.989433,6.111611,6.092421,6.281226,6.040389,6.131345,6.635692,6.061870,⋯,6.112284,6.114066,6.248340,6.151032,6.232299,5.966215,6.321657,6.060676,6.077992,BCAN
42313,42313,91952_at,6.749391,6.489252,6.394461,6.695105,6.420493,6.905075,6.722545,6.580546,⋯,7.029894,7.476906,7.291986,6.929070,7.116279,6.582520,6.431906,6.457010,6.030307,DCAF15


In [19]:
expr.mat = expr.mat[!duplicated(expr.mat$name),]

In [20]:
dim(expr.mat)

[1] 20660   467

In [21]:
tail(expr.mat)

,X,ID,GSM1254265.CEL,GSM1254267.CEL,GSM1254268.CEL,GSM1254269.CEL,GSM1254270.CEL,GSM1254271.CEL,GSM1254272.CEL,GSM1254273.CEL,⋯,GSM726249.CEL,GSM726251.CEL,GSM726252.CEL,GSM726253.CEL,GSM726256.CEL,GSM726257.CEL,GSM726258.CEL,GSM726259.CEL,GSM726260.CEL,name
,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
42236,42236,57739_at,7.253950,7.236794,5.995227,6.603600,6.893824,6.861632,5.891815,6.986671,⋯,6.863166,7.226329,6.006122,6.535600,7.311797,6.541876,6.737183,7.020209,7.355203,DND1P1
42266,42266,64408_s_at,5.975910,4.020682,5.394315,5.085051,5.497240,7.694316,6.324222,6.712675,⋯,6.178140,5.865860,5.505338,4.240999,4.679240,5.610164,6.537012,6.023501,4.706747,64408_s_at
42275,42275,64900_at,3.074506,2.837487,2.773762,2.760590,2.741710,3.048127,3.078253,2.907534,⋯,3.348545,3.535437,3.084098,4.230598,2.816035,3.409068,2.921560,3.368075,2.706996,TMEM231
42298,42298,81737_at,5.782047,5.998033,5.723267,6.003600,5.948172,6.099029,5.174717,5.317616,⋯,5.847827,5.760503,5.763633,5.436408,5.052692,5.873616,5.685748,5.781354,5.832693,81737_at
42301,42301,89476_r_at,6.884389,6.466569,6.485259,6.750198,6.706266,7.183014,7.329092,6.932983,⋯,7.016904,7.179459,6.755633,6.982025,7.207387,6.942184,7.102979,6.938517,6.874608,STX16-NPEPL1
42305,42305,90610_at,9.224288,8.821587,9.134468,8.010974,9.504773,9.739866,8.904690,9.337613,⋯,8.849445,9.631514,8.454110,8.295130,9.539139,8.894052,9.352746,9.303458,9.373897,90610_at


In [22]:
expr.mat = expr.mat[,!(names(expr.mat) %in% c("X", "ID"))]

In [23]:
head(expr.mat)

,GSM1254265.CEL,GSM1254267.CEL,GSM1254268.CEL,GSM1254269.CEL,GSM1254270.CEL,GSM1254271.CEL,GSM1254272.CEL,GSM1254273.CEL,GSM1254274.CEL,GSM1254275.CEL,⋯,GSM726249.CEL,GSM726251.CEL,GSM726252.CEL,GSM726253.CEL,GSM726256.CEL,GSM726257.CEL,GSM726258.CEL,GSM726259.CEL,GSM726260.CEL,name
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>
1,7.537840,6.800871,6.752669,5.929756,7.353971,6.496232,6.716364,7.407105,7.955677,6.978522,⋯,7.605745,7.701945,7.096387,6.600813,6.563042,6.671309,5.930616,6.119624,6.554417,RFC2
2,4.679401,4.777950,4.360962,4.509553,4.720435,7.669224,5.432612,4.435121,4.860942,6.589701,⋯,4.396333,5.979429,4.177423,4.290783,5.271843,5.841040,7.307721,6.138064,4.855038,HSPA6
3,6.798042,7.370561,7.043800,7.132901,7.272406,6.851993,7.215697,7.225802,7.017084,7.035230,⋯,7.564590,6.726327,7.424036,7.026728,6.710751,7.198889,6.996386,6.894396,7.371354,PAX8
4,2.704549,2.500655,2.644052,2.712234,2.861128,2.913571,2.820453,2.752378,2.886452,2.818774,⋯,2.650575,2.795106,2.576289,2.807488,2.784839,2.746156,2.694078,3.009329,2.856732,GUCA1A
5,4.544424,5.050274,4.988216,4.862662,5.144750,4.837291,4.871102,4.981282,4.780484,4.604664,⋯,4.846550,5.505493,5.088999,5.112156,5.743054,5.497159,5.444714,5.167476,5.273593,THRA
6,3.734027,3.630663,3.587355,4.006397,3.762099,3.608655,3.754732,3.931028,3.686961,3.656161,⋯,3.784949,3.751392,3.702852,3.803101,3.597913,3.614316,3.799544,3.681782,3.887242,PTPN21


In [24]:
write.csv(expr.mat, "GSE147515_FAB_mat.csv")